## Extract all fields and check which aren't mapped or resolved

### Usage

Run All

3 files are written to disk to help with the mapping:

`./countryData/v1Forms.json`

`./countryData/v1FormFields.json`

`./countryData/v2EventFields.json`


Output will give 4 lists for each form:

#### Unmapped V1 form fields
Fields in the v1 form which do not have a mapping in v2

Add field to the appropriate mapping file:

For names: `./countryData/nameMappings.ts`

Address: `./countryData/addressMappings.ts`

Everything else: `./countryData/countryMappings.ts`


#### V1 form fields mapped to a V2 field that does not exist
You have an incorrect mapping, check the json files to make sure you have the correct v2 field name.

#### V1 form fields mapped but V2 field does not have a resolver

The field is mapped but isn't resolved

Add field to the appropriate resolver file:

For names: `./countryData/nameResolver.ts`

Address: `./countryData/addressResolver.ts`

Everything else: `./countryData/countryResolvers.ts`

#### Additional V2 form fields not mapped

Fields in the v2 form which do not exist in v1

These are not necessarily errors but may indicate that you've created some new fields in the v2 form. It's worth checking these.




In [ ]:
import { REGISTRAR_USERNAME, REGISTRAR_PASSWORD } from './helpers/vars.ts'
import { authenticate } from './helpers/authentication.ts'

const token = await authenticate(REGISTRAR_USERNAME, REGISTRAR_PASSWORD)
token


In [ ]:
import { fetchEvents, fetchForms } from './helpers/formsHandlers.ts'

const forms = await fetchForms(token)
const events = await fetchEvents(token)

Deno.writeFileSync(
  './countryData/v1Forms.json',
  new TextEncoder().encode(JSON.stringify(forms, null, 2)))

In [ ]:
import { extractFieldType, extractFormFields } from './helpers/utils.ts'

const ignoredFields = [
  'DIVIDER',
  'PARAGRAPH',
  'BULLET_LIST',
  'HEADING3',
  'SIGNATURE',
]

const birthFormFields = extractFormFields(forms, 'birth')
  .filter((x) => !ignoredFields.includes(x.type))
  .map((f) => f.id)

const deathFormFields = extractFormFields(forms, 'death')
  .filter((x) => !ignoredFields.includes(x.type))
  .map((f) => f.id)

const birthEvent = events.find((x) => x.id === 'birth')
const deathEvent = events.find((x) => x.id === 'death')

const birthEventFields = [
  ...new Set(
    extractFieldType(birthEvent, 'fields')
      .filter((x) => !ignoredFields.includes(x.type))
      .map((f) => f.id)
      .filter((x) => x)
  ),
]

const deathEventFields = [
  ...new Set(
    extractFieldType(deathEvent, 'fields')
      .filter((x) => !ignoredFields.includes(x.type))
      .map((f) => f.id)
      .filter((x) => x)
  ),
]

Deno.writeFileSync(
  './countryData/v1FormFields.json',
  new TextEncoder().encode(JSON.stringify([birthFormFields, deathFormFields], null, 2)))
Deno.writeFileSync(
  './countryData/v2EventFields.json',
  new TextEncoder().encode(JSON.stringify([birthEventFields, deathEventFields], null, 2)))


In [ ]:
import {
  DEFAULT_FIELD_MAPPINGS,
  CUSTOM_FIELD_MAPPINGS,
} from './helpers/defaultMappings.ts'
import { ADDRESS_MAPPINGS } from './countryData/addressMappings.ts'
import { NAME_MAPPINGS } from './countryData/nameMappings.ts'
import { COUNTRY_FIELD_MAPPINGS } from './countryData/countryMappings.ts'
import defaultResolver, {
  defaultBirthResolver,
  defaultDeathResolver,
} from './helpers/defaultResolvers.ts'
import { countryResolver } from './countryData/countryResolvers.ts'
import historyResolver from './helpers/historyResolver.ts'
import { v1IgnoreList } from './countryData/ignoreList.ts'

const birthResolver = {...defaultBirthResolver, ...defaultResolver, ...countryResolver }
const deathResolver = { ...defaultDeathResolver, ...defaultResolver, ...countryResolver }

const addressMap = Object.fromEntries(
  Object.entries(ADDRESS_MAPPINGS).map(([f, value]) => [
    f,
    Object.keys(value(null))[0],
  ])
)

const nameMap = Object.fromEntries(
  Object.entries(NAME_MAPPINGS).map(([f, value]) => [
    f,
    Object.keys(value(null))[0],
  ])
)

const allMappings = {
  ...DEFAULT_FIELD_MAPPINGS,
  ...CUSTOM_FIELD_MAPPINGS,
  ...addressMap,
  ...nameMap,
  ...COUNTRY_FIELD_MAPPINGS,
}

function partition<T>(list: T[], predicate: (item: T) => boolean) {
  const map: T = {}
  const unmapped: T[] = []

  for (const i of list) {
    if (predicate(i)) {
      map[i] = predicate(i)
    } else {
      unmapped.push(i)
    }
  }

  return { map, unmapped }
}

const birthFormMappings = partition(
  birthFormFields.filter((f) => !v1IgnoreList.includes(f)),
  (f) => allMappings[f]
)
const deathFormMappings = partition(
  deathFormFields.filter((f) => !v1IgnoreList.includes(f)),
  (f) => allMappings[f]
)

const unMappedBirthEventFields = birthEventFields.filter(
  (f) =>
    !Object.values(birthFormMappings.map).includes(f) &&
    !Object.keys(historyResolver).includes(f)
)
const unMappedDeathEventFields = deathEventFields.filter(
  (f) =>
    !Object.values(deathFormMappings.map).includes(f) &&
    !Object.keys(historyResolver).includes(f)
)

const badBirthMappings = Object.entries(birthFormMappings.map).filter(
  ([_k, v]) => !birthEventFields.includes(v)
)

const badDeathMappings = Object.entries(deathFormMappings.map).filter(
  ([_k, v]) => !deathEventFields.includes(v)
)

const unmappedFields = {
  birth: {
    missingV1Mappings: birthFormMappings.unmapped,
    extraV2Mappings: unMappedBirthEventFields,
  },
  death: {
    missingV1Mappings: deathFormMappings.unmapped,
    extraV2Mappings: unMappedDeathEventFields,
  },
}

const unResolvedBirthFields = Object.entries(birthFormMappings.map)
  .filter(([_k, v]) => !badBirthMappings.some(([_k1, v1]) => v1 === v))
  .filter(([_k, v]) => !birthResolver[v])

const unResolvedDeathFields = Object.entries(deathFormMappings.map)
  .filter(([_k, v]) => !badDeathMappings.some(([_k1, v1]) => v1 === v))
  .filter(([_k, v]) => !deathResolver[v])

In [ ]:
const render = (list) => {
  if (list.length) {
    console.table(list)
  } else {
    console.log('None \x1b[32m✔\x1b[0m')
  }
}

console.log('Unmapped V1 form fields:')
console.log()

console.log('Birth:')
render(unmappedFields.birth.missingV1Mappings)

console.log('Death:')
render(unmappedFields.death.missingV1Mappings)
console.log()
console.log('V1 form fields mapped to a V2 field that does not exist:')
console.log('Birth:')
render(badBirthMappings.map(([k, v]) => ({ 'V1 Field': k, 'V2 Field': v })))

console.log('Death:')
render(badDeathMappings.map(([k, v]) => ({ 'V1 Field': k, 'V2 Field': v })))

console.log()
console.log('V1 form fields mapped but V2 field does not have a resolver:')
console.log('Birth:')
render(
  unResolvedBirthFields.map(([k, v]) => ({ 'V1 Field': k, 'V2 Field': v }))
)
console.log('Death:')
render(
  unResolvedDeathFields.map(([k, v]) => ({ 'V1 Field': k, 'V2 Field': v }))
)

console.log()
console.log('Additional V2 form fields not mapped:')
console.log('Birth:')
render(unmappedFields.birth.extraV2Mappings)

console.log('Death:')
render(unmappedFields.death.extraV2Mappings)

if (
  unmappedFields.birth.missingV1Mappings.length ||
  unmappedFields.death.missingV1Mappings.length ||
  badBirthMappings.length ||
  badDeathMappings.length ||
  unResolvedBirthFields.length ||
  unResolvedDeathFields.length
) {
  throw new Error('Migration may not proceed. Please fix errors')
} else {
  console.log()
  console.log('Ok to proceed with migration')
}
